# INDIAN STARTUP FUNDING

![](https://www.theindianwire.com/wp-content/uploads/2020/05/Startup.jpg)

Questions to be analyzed-
<ul>
    <li>No of startups funded each year and Year-wise Fund Allocated</li>
    <li>Total funding in each year</li>
    <li>City which got the maximum no of startups funded</li>
    <li>City in which startups receieved highest funding</li>
    <li>Percentage Share of each Industry vertical</li>
    <li>Type of Investment V/s Funding received</li>
    <li>Investor V/s Amount Spent</li>
    <li>Investor V/s No. Of Startups Funded</li>
    <li>Startups V/s Fund Allocated</li>
</ul>

In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv('/kaggle/input/indian-startup-funding/startup_funding.csv')
df

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
df.rename(columns={'Date dd/mm/yyyy':'Date','City  Location':'City','InvestmentnType':'InvestmentType','Amount in USD':'Amount(in USD)','Investors Name':'Investors'},inplace=True)
df

In [ ]:
df.drop(columns='Sr No',inplace=True)

In [ ]:
df=df.replace(r'\\\\xc2\\\\xa0','')

# Cleaning dates

In [ ]:
df[df['Date'].str.contains('(\d{2})[/](\d{2})[/](\d{4})')==False]

In [ ]:
import datetime
def correct_date(x,y):
    if(len(y[x.str.contains(".")==True])>0):
        x=x.str.replace(".","/")
    if(len(y[x.str.contains("\\xc2\\xa0")==True])>0):
        x=x.str.replace("\\xc2\\xa0","")
    if(len(y[x.str.contains("//")==True])>0):
        x=x.str.replace("//","/")
    if(len(y[x.str.contains("-")==True])>0):
        x=x.str.replace("-","/")
    if(len(y[x.str.contains(r"\\\\xc2\\\\xa0")==True])>0):
        x=x.str.replace(r"\\\\xc2\\\\xa0","")
    return x

df['Date']=correct_date(df['Date'],df)
df.loc[192,"Date"]="05/07/2018"
df.loc[2571,"Date"]="01/07/2015"
df.loc[2606,"Date"]="10/7/2015"
df['Date']=df['Date'].apply(lambda x: datetime.datetime.strptime(x, '%d/%m/%Y'))

"""
ind1=df[df['Date'].str.contains('05/072018')==True].index
df.loc[ind1,'Date']='05/07/2018'
df.loc[2606,'Date']='10/07/2015'
df[df['Date'].str.contains('(\d{2})[/](\d{2})[/](\d{4})')==False]
ind3=df[df['Date'].str.contains('12/05.2015')==True].index
df.loc[ind3,'Date']='12/05/2015'
ind4=df[df['Date'].str.contains('13/04.2015')==True].index
df.loc[ind4,'Date']='13/04/2015'
ind5=df[df['Date'].str.contains('15/01.2015')==True].index
df.loc[ind5,'Date']='15/01/2015'
ind6=df[df['Date'].str.contains('22/01//2015')==True].index
df.loc[ind6,'Date']='22/01/2015'
ind7=df[df['Date'].str.contains('01/07/015')==True].index
df.loc[ind7,'Date']='01/07/2015'
df[df['Date'].str.contains('(\d{2})[/](\d{2})[/](\d{4})')==False]
"""

df.info()

# Cleaning Amount

In [ ]:
df['Amount(in USD)']=df['Amount(in USD)'].str.replace(",","")
def correct_amount(x,y):
    """
    if(len(y[x.str.contains("undisclosed")==True])>0):
        x=x.str.replace("undisclosed","")
    if(len(y[x.str.contains(r"\\xc2\\xa0")==True])>0):
        x=x.str.replace(r"\\xc2\\xa0","")
    if(len(y[x.str.contains("unknown")==True])>0):
        x=x.str.replace("unknown","")
    if(len(y[x.str.contains("Undisclosed")==True])>0):
        x=x.str.replace("Undisclosed","")
    """
    if(len(y[x.str.contains(r"\\xc2\\xa0N/A")==True])>0):
        x=x.str.replace(r"\\xc2\\xa0N/A","")
    if(len(y[x.str.contains(r'\\\\xc2\\\\xa0')==True])>0):
        x=x.str.replace(r'\\\\xc2\\\\xa0',"")
    if(len(y[x.str.contains(r'N/A')==True])>0):
        x=x.str.replace(r'\\\\xc2\\\\xa0',"")
    return x
df['Amount(in USD)']=correct_amount(df['Amount(in USD)'],df)
df.loc[109,"Amount(in USD)"]=df.loc[109,"Amount(in USD)"][0:-1]

In [ ]:
df.loc[df['Amount(in USD)'].isin(['undisclosed','unknown','Undisclosed',r"\\xc2\\xa0N/A","N/A"]),'Amount(in USD)']=np.NaN

In [ ]:
df[df['Amount(in USD)'].str.contains('[^0-9]')==True]

In [ ]:
df['Amount(in USD)']=np.round(df['Amount(in USD)'].astype(float))
df.info()

# Cleaning Startup Name

In [ ]:
df['Startup Name'].head()

In [ ]:
li=list(df[df['Startup Name'].str.contains('[0-9]',regex=True)==True]["Startup Name"])

In [ ]:
def correct_name(x,y):
    if(len(y[x.str.contains(r'\\\\xc2\\\\xa0')==True])>0):
        x=x.str.replace(r'\\\\xc2\\\\xa0',"")
    if(len(y[x.str.contains(r'https://www.wealthbucket.in/')==True])>0):
        x=x.str.replace(r'https://www.wealthbucket.in/',"wealthbucket")
    if(len(y[x.str.contains(r'\\\\xe2\\\\x80\\\\x99')==True])>0):
        x=x.str.replace(r'\\\\xe2\\\\x80\\\\x99',"")
    if(len(y[x.str.contains(r'\\xe2\\x80\\x99')]==True)>0):
        x=x.str.replace(r'\\xe2\\x80\\x99','')
    return x
df['Startup Name']=correct_name(df['Startup Name'],df)

In [ ]:
li_=list(df[df['Startup Name'].str.contains('[0-9]',regex=True)==True]["Startup Name"])

# Cleaning Variable City

In [ ]:
ci=list(df['City'].value_counts().index)

In [ ]:
#Since we will deal with only Indian States & Cities, We will replace all the cities where multiple cities are given with first Indian City/state
def correct_city(x,y):
    if(len(y[x.str.contains(r'\\\\xc2\\\\xa0')==True])>0):
        x=x.str.replace(r'\\\\xc2\\\\xa0',"")
    if(len(y[x.str.contains(r'\\\\\\\\xc2\\\\\\\\xa0')==True])>0):
        x=x.str.replace(r'\\\\\\\\xc2\\\\\\\\xa0',"")
    #Bengaluru
    if(len(y[x.str.contains('Bangalore')==True])>0):
        x=x.str.replace('Bangalore',"Bengaluru")
    if(len(y[x.str.contains('Bangalore')==True])>0):
        x=x.str.replace('Bangalore',"Bengaluru")
    if(len(y[x.str.contains(r'Bengaluru and Gurugram')==True])>0):
        x=x.str.replace(r'Bengaluru and Gurugram','Bengaluru')
    if(len(y[x.str.contains(r'Bengaluru/ Bangkok')==True])>0):
        x=x.str.replace(r'Bengluru/ Bangkok','Bengaluru')
    if(len(y[x.str.contains(r'Bengaluru / SFO')==True])>0):
        x=x.str.replace(r'Bengaluru / SFO','Bengaluru')
    if(len(y[x.str.contains(r'SFO / Bengaluru')==True])>0):
        x=x.str.replace(r'SFO / Bengaluru','Bengaluru')
    if(len(y[x.str.contains(r'Bengaluru/ Bangkok')==True])>0):
        x=x.str.replace(r'Bengaluru/ Bangkok','Bengaluru')
    if(len(y[x.str.contains(r'New York, Bengaluru')==True])>0):
        x=x.str.replace(r'New York, Bengaluru','Bengaluru')
    if(len(y[x.str.contains(r'Bengaluru / Palo Alto')==True])>0):
        x=x.str.replace(r'Bengaluru / Palo Alto','Bengaluru')
    if(len(y[x.str.contains(r'Bengaluru / San Mateo')==True])>0):
        x=x.str.replace(r'Bengaluru / San Mateo','Bengaluru')
    if(len(y[x.str.contains(r'Seattle / Bengaluru')==True])>0):
        x=x.str.replace(r'Seattle / Bengaluru','Bengaluru')
    if(len(y[x.str.contains(r'Bengaluru / USA')==True])>0):
        x=x.str.replace(r'Bengaluru / USA','Bengaluru')   
    #Gurgaon
    if(len(y[x.str.contains("Gurgaon")==True])>0):
        x=x.str.replace('Gurgaon','Gurugram')
    if(len(y[x.str.contains(r'Gurugram / SFO')==True])>0):
        x=x.str.replace(r'Gurugram / SFO','Bengaluru')
    #Delhi
    if(len(y[x.str.contains(r'Nw Delhi')==True])>0):
        x=x.str.replace('Nw Delhi','Delhi')
    if(len(y[x.str.contains(r'Delhi / California')==True])>0):
        x=x.str.replace('Delhi / California','Delhi')
    if(len(y[x.str.contains(r'Delhi / Houston')==True])>0):
        x=x.str.replace(r'Delhi / Houston','Delhi')
    if(len(y[x.str.contains(r'New Delhi')==True])>0):
        x=x.str.replace(r'New Delhi','Delhi')
    if(len(y[x.str.contains(r'Delhi / US')==True])>0):
        x=x.str.replace(r'Delhi / US','Delhi')
    if(len(y[x.str.contains(r'Delhi & Cambridge')==True])>0):
        x=x.str.replace(r'Delhi & Cambridge','Delhi')
    if(len(y[x.str.contains(r'Delhi/ Houston')==True])>0):
        x=x.str.replace(r'Delhi/ Houston','Delhi')
    
    #Goa
    if(len(y[x.str.contains(r'Goa/Hyderabad')==True])>0):
        x=x.str.replace(r'Goa/Hyderabad','Goa')
    if(len(y[x.str.contains(r'Panaji')==True])>0):
        x=x.str.replace(r'Panaji','Goa')
    #Mumbai
    if(len(y[x.str.contains(r'Mumbai / Global')==True])>0):
        x=x.str.replace(r'Mumbai / Global','Mumbai')
    if(len(y[x.str.contains(r'Andheri')==True])>0):
        x=x.str.replace(r'Andheri','Mumbai')
    if(len(y[x.str.contains(r'Mumbai/Bengaluru')==True])>0):
        x=x.str.replace(r'Mumbai/Bengaluru','Mumbai')
    if(len(y[x.str.contains(r'Mumbai / UK')==True])>0):
        x=x.str.replace(r'Mumbai / UK','Mumbai')
    if(len(y[x.str.contains(r'Mumbai / NY')==True])>0):
        x=x.str.replace(r'Mumbai / NY','Mumbai')
    #Pune
    if(len(y[x.str.contains(r'Pune / Singapore')==True])>0):
        x=x.str.replace(r'Pune / Singapore','Pune')
    if(len(y[x.str.contains(r'Pune / US')==True])>0):
        x=x.str.replace(r'Pune / US','Pune')
    if(len(y[x.str.contains(r'Pune / Dubai')==True])>0):
        x=x.str.replace(r'Pune / Dubai','Pune')
    if(len(y[x.str.contains(r'Pune/Seattle')==True])>0):
        x=x.str.replace(r'Pune/Seattle','Pune')
    #Hyderabad
    if(len(y[x.str.contains(r'Dallas / Hyderabad')==True])>0):
        x=x.str.replace(r'Dallas / Hyderabad','Hyderabad')
    if(len(y[x.str.contains(r'Hyderabad/USA')==True])>0):
        x=x.str.replace(r'Hyderabad/USA','Hyderabad')
    #Chennai
    if(len(y[x.str.contains(r'Chennai/ Singapore')==True])>0):
        x=x.str.replace(r'Chennai/ Singapore','Chennai')
    #Noida
    if(len(y[x.str.contains(r'Noida / Singapore')==True])>0):
        x=x.str.replace(r'Noida / Singapore','Noida')
    return x

df['City']=correct_city(df['City'],df)


# Cleaning Industry Vertical

In [ ]:
df[df['Industry Vertical'].str.contains('[0-9]',regex=True)==True]

In [ ]:
def correct_industry(x,y):
    if(len(y[x.str.contains(r'\\\\xc2\\\\xa0')==True])>0):
        x=x.str.replace(r'\\\\xc2\\\\xa0',"")
    if(len(y[x.str.contains(r'\\\\xc3\\\\xa9')==True])>0):
        x=x.str.replace(r'\\\\xc3\\\\xa9',"")
    if(len(y[x.str.contains(r'\\\\xe2\\\\x80\\\\x99s')==True])>0):
        x=x.str.replace(r'\\\\xe2\\\\x80\\\\x99',"")
    if(len(y[x.str.contains(r'\\xe2\\x80\\x93')==True])>0):
        x=x.str.replace(r'\\xe2\\x80\\x93',"")   
    if(len(y[x.str.contains(r'\\\\n')==True])>0):
        x=x.str.replace(r'\\\\n'," ")   
    if(len(y[x.str.contains(r'EdTech')==True])>0):
        x=x.str.replace(r'EdTech',"Ed-Tech")   
    if(len(y[x.str.contains(r'Edtech')==True])>0):
        x=x.str.replace(r'Edtech',"Ed-Tech")
    if(len(y[x.str.contains(r'Ecommerce')==True])>0):
        x=x.str.replace(r'Ecommerce',"E-commerce")
    if(len(y[x.str.contains(r'eCommerce')==True])>0):
        x=x.str.replace(r'eCommerce',"E-commerce")
    if(len(y[x.str.contains(r'ecommerce')==True])>0):
        x=x.str.replace(r'ecommerce',"E-commerce")
    if(len(y[x.str.contains(r'ECommerce')==True])>0):
        x=x.str.replace(r'ECommerce',"E-commerce")
    if(len(y[x.str.contains(r'E-Commerce')==True])>0):
        x=x.str.replace(r'E-Commerce',"E-commerce")
    if(len(y[x.str.contains(r'Fintech')==True])>0):
        x=x.str.replace(r'Fintech',"FinTech")
    if(len(y[x.str.contains(r'Fin-Tech')==True])>0):
        x=x.str.replace(r'Fin-Tech',"FinTech")
    if(len(y[x.str.contains(r'healthcare')==True])>0):
        x=x.str.replace(r'healthcare',"Healthcare")
    if(len(y[x.str.contains(r'SAAS')==True])>0):
        x=x.str.replace(r'SAAS',"SaaS")
    if(len(y[x.str.contains(r'Saas')==True])>0):
        x=x.str.replace(r'Saas',"SaaS")
    if(len(y[x.str.contains(r'logistics')==True])>0):
        x=x.str.replace(r'logistics',"Logistics")
    if(len(y[x.str.contains(r'OnDemand')==True])>0):
        x=x.str.replace(r'OnDemand',"OnDemand")
    if(len(y[x.str.contains(r'on-demand')==True])>0):
        x=x.str.replace(r'on-demand',"OnDemand")
    if(len(y[x.str.contains(r'online')==True])>0):
        x=x.str.replace(r'online',"Online")
    
    return x

df['Industry Vertical']=correct_industry(df['Industry Vertical'],df)

# Cleaning Industry Sub-vertical

In [ ]:
def correct_subvert(x,y):
    if(len(y[x.str.contains(r'Online lending platform')==True])>0):
        x=x.str.replace(r'Online lending platform',"Online Lending Platform")
    if(len(y[x.str.contains(r'Online Lending')==True])>0):
        x=x.str.replace(r'Online Lending',"Online Lending Platform")
    if(len(y[x.str.contains(r'\\\\xe2\\\\x80\\\\x99')==True])>0):
        x=x.str.replace(r'\\\\xe2\\\\x80\\\\x99',"")
    if(len(y[x.str.contains(r'\\\\\\\\xe2\\\\\\\\x80\\\\\\\\x99')==True])>0):
        x=x.str.replace(r'\\\\\\\\xe2\\\\\\\\x80\\\\\\\\x99',"")
    if(len(y[x.str.contains(r'\\xe2\\x80\\x99')==True])>0):
        x=x.str.replace(r'\\xe2\\x80\\x99',"")
    if(len(y[x.str.contains(r'\\\\xc2\\\\xa0')==True])>0):
        x=x.str.replace(r'\\\\xc2\\\\xa0',"")
    if(len(y[x.str.contains(r'\\\\xc3\\\\xa9')==True])>0):
        x=x.str.replace(r'\\\\xc3\\\\xa9',"")
    
    return x
df['SubVertical']=correct_subvert(df['SubVertical'],df)

# Cleaning Investors Name

In [ ]:
def correct_investor(x,y):
    if(len(y[x.str.contains(r'\\\\xc2\\\\xa0')==True])>0):
        x=x.str.replace(r'\\\\xc2\\\\xa0',"")
    if(len(y[x.str.contains(r'\\\\\\\\xc2\\\\\\\\xa0')==True])>0):
        x=x.str.replace(r'\\\\\\\\xc2\\\\\\\\xa0',"")
    if(len(y[x.str.contains(r'\\\\xe2\\\\x80\\\\x99O')==True])>0):
        x=x.str.replace(r'\\\\xe2\\\\x80\\\\x99O',"")
    if(len(y[x.str.contains(r'\\\\\\\\xe2\\\\\\\\x80\\\\\\\\x99O')==True])>0):
        x=x.str.replace(r'\\\\\\\\xe2\\\\\\\\x80\\\\\\\\x99O',"")
    if(len(y[x.str.contains(r'\\\xe2\\\\x80\\\\x99')==True])>0):
        x=x.str.replace(r'\\\xe2\\\\x80\\\\x99',"")
    if(len(y[x.str.contains(r'\\\\xe2\\\\x80\\\\x99')==True])>0):
        x=x.str.replace(r'\\\\xe2\\\\x80\\\\x99',"")
    if(len(y[x.str.contains(r'\\\\\\\\xc3\\\\\\\\x98')==True])>0):
        x=x.str.replace(r'\\\\\\\\xc3\\\\\\\\x98',"")
    if(len(y[x.str.contains(r'\\\\\\\\n\\\\\\\\n')==True])>0):
        x=x.str.replace(r'\\\\\\\\n\\\\\\\\n',"")

    if(len(y[x.str.contains(r'Undisclosed Investor')==True])>0):
        x=x.str.replace(r'Undisclosed Investor',"Undisclosed Investors")
    if(len(y[x.str.contains(r'Undisclosed investor')==True])>0):
        x=x.str.replace(r'Undisclosed investor',"Undisclosed Investors")
    if(len(y[x.str.contains(r'Undisclosed Investorss')==True])>0):
        x=x.str.replace(r'Undisclosed Investorss',"Undisclosed Investors")
    if(len(y[x.str.contains(r'Undisclosed HNIs')==True])>0):
        x=x.str.replace(r'Undisclosed HNIs',"Undisclosed Investors")
    if(len(y[x.str.contains(r'Undisclosed Angel investors & HNIs')==True])>0):
        x=x.str.replace(r"Undisclosed Angel investors & HNIs","Undisclosed Investors")
    if(len(y[x.str.contains(r'undisclosed investors')==True])>0):
        x=x.str.replace(r"undisclosed investors","Undisclosed Investors")
    if(len(y[x.str.contains(r'undisclosed investor')==True])>0):
        x=x.str.replace(r"undisclosed investor","Undisclosed Investors")
    if(len(y[x.str.contains(r'Undisclosed angel investors')==True])>0):
        x=x.str.replace(r"Undisclosed angel investors","Undisclosed Investors")
    if(len(y[x.str.contains(r'High Networth Individuals (undisclosed)')==True])>0):
        x=x.str.replace(r"High Networth Individuals (undisclosed)","Undisclosed Investors")
    if(len(y[x.str.contains(r'undisclosed private investors')==True])>0):
        x=x.str.replace(r"undisclosed private investors","Undisclosed Investors")
    if(len(y[x.str.contains(r'SoftBank Group')==True])>0):
        x=x.str.replace(r"SoftBank Group","SoftBank")
    if(len(y[x.str.contains(r'Softbank')==True])>0):
        x=x.str.replace(r"Softbank","SoftBank")
    
    return x
df['Investors']=correct_investor(df['Investors'],df)
df['Investors']=df['Investors'].str.replace(r' . ',",")
df['Investors']=df['Investors'].str.replace(r' and ',",")
df['Investors']=df['Investors'].str.replace(r' & ',",")
df['Investors']=df['Investors'].str.strip()
df['InvestmentType']=df['InvestmentType'].str.strip()
df['Startup Name']=df['Startup Name'].str.strip()
df['City']=df['City'].str.strip()
df['Industry Vertical']=df['Industry Vertical'].str.strip()
df['SubVertical']=df['SubVertical'].str.strip()

# Cleaning Investment Type


In [ ]:
df['InvestmentType'].value_counts()

In [ ]:
def correct_funding(x,y):
    #Seed-Angel Funding
    if(len(y[x.str.contains(r'Seed/ Angel Funding')==True])>0):
        x=x.str.replace(r'Seed/ Angel Funding',"Seed Angel Funding")
    if(len(y[x.str.contains(r'Angel / Seed Funding')==True])>0):
        x=x.str.replace(r'Angel / Seed Funding',"Seed Funding")
    if(len(y[x.str.contains(r'Seed / Angel Funding')==True])>0):
        x=x.str.replace(r'Seed / Angel Funding',"Seed Angel Funding")
    if(len(y[x.str.contains(r'Seed / Angle Funding')==True])>0):
        x=x.str.replace(r'Seed / Angle Funding',"Seed Funding")
    if(len(y[x.str.contains(r'Seed/Angel Funding')==True])>0):
        x=x.str.replace(r'Seed/Angel Funding',"Seed Funding")
    #Seed Funding
    if(len(y[x.str.contains(r'Seed\\\\nFunding')==True])>0):
        x=x.str.replace(r'Seed\\\\nFunding',"Seed Funding")
    if(len(y[x.str.contains(r'Seed\\nFunding')==True])>0):
        x=x.str.replace(r'Seed\\nFunding',"Seed Funding")
    if(len(y[x.str.contains(r'Seed Round')==True])>0):
        x=x.str.replace(r'Seed Round',"Seed Funding")
    if(len(y[x.str.contains(r'Seed funding')==True])>0):
        x=x.str.replace(r'Seed funding',"Seed Funding")
    if(len(y[x.str.contains(r'Seed Funding Round')==True])>0):
        x=x.str.replace(r'Seed Funding Round',"Seed Funding")
    #Pre-Series A
    if(len(y[x.str.contains(r'pre-series A')==True])>0):
        x=x.str.replace(r'pre-series A',"Pre-Series A")
    if(len(y[x.str.contains(r'Pre Series A')==True])>0):
        x=x.str.replace(r'Pre Series A',"Pre-Series A")
    if(len(y[x.str.contains(r'Pre-series A')==True])>0):
        x=x.str.replace(r'Pre-series A',"Pre-Series A")
    if(len(y[x.str.contains(r'pre-Series A')==True])>0):
        x=x.str.replace(r'pre-Series A',"Pre-Series A")
    #Equity
    if(len(y[x.str.contains(r'Equity Based Funding')==True])>0):
        x=x.str.replace(r'Equity Based Funding',"Equity")
    #Angel
    if(len(y[x.str.contains(r'Angel Funding')==True])>0):
        x=x.str.replace(r'Angel Funding',"Angel")
    if(len(y[x.str.contains(r'Angel Round')==True])>0):
        x=x.str.replace(r'Angel Round',"Angel")
    #Private Equity
    if(len(y[x.str.contains(r'PrivateEquity')==True])>0):
        x=x.str.replace(r'PrivateEquity',"Private Equity")
    if(len(y[x.str.contains(r'Private Funding')==True])>0):
        x=x.str.replace(r'Private Funding',"Private Equity")
    if(len(y[x.str.contains(r'Private\\\\nEquity')==True])>0):
        x=x.str.replace(r'Private\\\\nEquity',"Private Equity")
    if(len(y[x.str.contains(r'Private Equity Round')==True])>0):
        x=x.str.replace(r'Private Equity Round',"Private Equity")
    return x
df['InvestmentType']=correct_funding(df['InvestmentType'],df)

# EDA

 

In [ ]:
year_val=list(df['Date'].dt.year.value_counts().index.sort_values())
year_count=list(df['Date'].dt.year.value_counts()[year_val])

In [ ]:
df['Date'].max()
#The last date in the dataset is 13 Jan 2020

# 1. No. of Startups funded in India between 2015-2020(Jan.)

In [ ]:
#Analyzing year wise , how many startups got funded?
sns.set_theme(style="darkgrid")
sns.countplot(df['Date'].dt.year)
plt.xlabel('Year')
plt.ylabel('No. of Startups Funded')
plt.title('No of Startups Funded V/s Year')
k=0
for i in range(len(df['Date'].dt.year.value_counts())):
    plt.annotate(year_count[i],xy =(k,year_count[i]))
    k+=1

Findings-
<ul>
<li>Maximum No. of startups were funded in 2016 followed by 2015. </li>
<li>2020 saw minimum because the data is limited to Jan 2020</li>
</ul>

In [ ]:
amount=list(df.groupby(df['Date'].dt.year).sum()['Amount(in USD)'])
def currency_inword(x):
    if(len(str(int(np.round(x))))>9):
        return str(x)[0]+'.'+str(x)[1:3]+'B'
    elif(len(str(int(np.round(x))))==9):
        return str(x)[0:3]+'M'
curr=[]
for i in range(len(amount)):
    curr.append(currency_inword(amount[i]))
curr

# 2.  Amount allocated to startups with respect to the years

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(
    data=[go.Scatter(y=amount, line=dict(color="crimson"))],
    layout=dict(title=dict(text="Amount Funded V/s Year")),
    
)

fig.update_layout(
    title="Amount Funded V/s Year",
    xaxis_title="Year",
    yaxis_title="Amount(in USD)",
    xaxis = dict(
        tickmode = 'array',
        tickvals = [0, 1, 2, 3, 4, 5],
        ticktext = year_val)
    
)

fig.show()

<ul>
    <li>The maximum amount in funding 10.42B USD was in 2017 followed by 9.70B USD in 2019 with the lease in 2016 i.e. 3.82B USD</li>
    <li>The minimum amount in funding is in 2020 due to date being limited January. However, one thing is quite interesting, even though maxiumum number of startups got funded in 2016, but the amount is the least among all years(not considering 2020).</li>
</ul>

# 3. No. of Startups funded with respect to the cities

In [ ]:
fig1 = go.Figure(go.Bar(
    y = df['City'].value_counts()[:10].values,
    x = df['City'].value_counts()[:10].index
))
fig1.update_xaxes(showgrid=True,)
fig1.update_layout(
    title="No of Startups Funded V/s City",
    xaxis_title="City",
    yaxis_title="No of Startups Funded"
    
)

# 4. Fund Allocation with respect to the cities

**Maximum number of startups were founded in Bengaluru(854) followed by Mumbai(574) ,Delhi(463) ,Gurugram(341) and Pune(112).
The reasons for Bengalurur being the startup capital of India can be attributed to it being the technical hub with almost every MNC being located there.**

In [ ]:
fig2 = go.Figure(go.Bar(
    x = df.groupby('City')['Amount(in USD)'].sum().sort_values(ascending=False)[:7].index,
    y = df.groupby('City')['Amount(in USD)'].sum().sort_values(ascending=False)[:7].values
))
fig2.update_xaxes(showgrid=True)
fig2.update_layout(
    title="Amount(in USD) V/s City",
    xaxis_title="City",
    yaxis_title="Amount(in USD) Funded"
)
fig2.show()

# 5.Percentage share of Industry vertical of startups

In [ ]:
fig3 = go.Figure(
    data=[go.Pie(values=df['Industry Vertical'].value_counts()[:10].values,labels=df['Industry Vertical'].value_counts()[:10].index)])
fig3.show()
"""
fig2.update_xaxes(showgrid=True)
fig2.update_layout(
    title="Amount(in USD) V/s City",
    xaxis_title="City",
    yaxis_title="Amount(in USD) Funded"
)
fig2.show()
"""

**The Consumer Internet dominates the industry vertical with approximately 50% followed by Technology(~ 25%), E-Commerce(~ 16%), Healthcare(~ 3.7%), Finance(~ 3.3%), Logistics(~ 1.7%), Education(~ 1.3%)**

# 6. Amount Funded with respect to the Investment Type

In [ ]:
import plotly.graph_objects as go
fig4 = go.Figure(go.Bar(
    x = df.groupby('InvestmentType')['Amount(in USD)'].sum().sort_values(ascending=False)[:10].index,
    y = df.groupby('InvestmentType')['Amount(in USD)'].sum().sort_values(ascending=False)[:10].values
))
fig4.update_xaxes(showgrid=True)
fig4.update_layout(
    title="Amount(in USD) V/s Investment Type",
    xaxis_title="Investment Type",
    yaxis_title="Amount(in USD) Funded"
)
fig4.show()

**Findings-
Private equity contributed the most(~27 B USD) in the funding of startups followed by Series B, D and C.**

# 7. No of startups funded with respect to the Investment Type

In [ ]:
import plotly.graph_objects as go
fig5 = go.Figure(go.Bar(
    x = df['InvestmentType'].value_counts()[:10].index,
    y = df['InvestmentType'].value_counts()[:10].values
))
fig5.update_xaxes(showgrid=True)
fig5.update_layout(
    title="No of Startups Funded V/s Investment Type",
    xaxis_title="Investment Type",
    yaxis_title="No of Startups Funded"
)
fig5.show()

**Seed Funding was the major contributor in the startup ecosystem with respect to the number of startups funded, funding approximately 1428 startups.**

# 8. Amount Allocated by the Investors

In [ ]:
import plotly.graph_objects as go
fig6 = go.Figure(go.Bar(
    x = df.groupby('Investors')['Amount(in USD)'].sum().sort_values(ascending=False)[:5].index,
    y = df.groupby('Investors')['Amount(in USD)'].sum().sort_values(ascending=False)[:5].values
))
fig6.update_xaxes(showgrid=True)
fig6.update_layout(
    title="Amount(in USD) V/s Investor",
    xaxis_title="Investors",
    yaxis_title="Amount(in USD) Funded"
)
fig6.show()

**SoftBank is the major investor in the startup ecosystem contributing the most with ~3.92B USD**

# 9. No. of Startups funded with respect to the Investor

In [ ]:
investor={}
for i in range(len(df['Investors'])):
    x=str(df['Investors'][i]).split(',')
    for j in x:
        j=j.lstrip()
        if j not in investor:
            investor[j]=1
        else:
            investor[j]+=1

In [ ]:
d={k:v for k,v in sorted(investor.items(),key=lambda item:item[1],reverse=True)[:20]}
d_={}
li=['Others','','others']
for i in d:
    if i in li:
        pass
    else:
        d_[i]=d[i]
d_
fig7 = go.Figure(go.Bar(
    y = list(d_.values()),
    x = list(d_.keys())
))
fig7.update_xaxes(showgrid=True)
fig7.update_layout(
    title="No of Startups Funded V/s Investor",
    xaxis_title="Investor",
    yaxis_title="No of Startups Funded"
    
)
fig7.show()

**Undisclosed investors funded the most startups with respect to the other investors.
Accel Partners is on the second position followed by Sequoia Capital, Blume ventures, SAIF Partners.
**

# 10. Top 3 Startups with respect to the fund allocation

In [ ]:
df.loc[df['Startup Name'].str.contains('Rapido Bike Taxi')==True,'Amount(in USD)']=3.900000e+09/72.39
fig8 = go.Figure(go.Bar(
    x = df.groupby('Startup Name')['Amount(in USD)'].sum().sort_values(ascending=False)[:3].values,
    y = df.groupby('Startup Name')['Amount(in USD)'].sum().sort_values(ascending=False)[:3].index,
    orientation='h'
))
fig8.update_xaxes(showgrid=True)
fig8.update_layout(
    title="Startups V/s Amount Funded",
    yaxis_title="Startups",
    xaxis_title="Amount Funded"
    
)
fig8.show()


**Flipkart raised the maximum amount i.e. 4.05 B USD followed by Paytm and OLA**

**Thanks!**